In [99]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary

In [100]:
df = pd.read_parquet("~/Data/BOQ_data.parquet")

df.tail()

,A2ME,BEME,BEME_adj,BEME_ind,CEI,DATE,DEC_ME,DEC_SHROUT,Div,Div_ann,...,s2invt,s2rect,sale,sale_ind,sales_g,sga2s,spread_mean,std_turn,std_vol,suv
3568650,0.211263,0.114912,-0.746238,0.861151,-12.189408,2023-08-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000152,0.708916,2.250092e+07,-1.580970
3568651,0.211263,0.114912,-0.750723,0.865635,-11.613183,2023-09-30,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000133,0.465122,1.476294e+07,-0.371920
3568652,0.211263,0.114912,-0.731231,0.846143,-12.752123,2023-10-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000177,0.531342,1.686876e+07,0.809301
3568653,0.211263,0.114912,-0.731231,0.846143,-7.943286,2023-11-30,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000245,0.614813,1.954419e+07,0.278301
3568654,0.211263,0.114912,-0.731231,0.846143,-9.359996,2023-12-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000186,0.599491,1.905735e+07,0.967711


In [101]:
# For this strategy we only need beme (book to market ratio) and the typical variables (permno, date, prc, ret)
keep = ['DATE', 'PERMNO', 'BEME', 'BEME_adj', 'PRC', 'RET']
df = df[keep]

# I like my variable lowercase and snake_case
df = df.rename(columns={'DATE':'caldt','PERMNO':'permno','BEME':'beme', 'BEME_adj':'beme_adj', 'PRC':'prc', 'RET': 'ret'})

# It's good practice to sort your data by permno and date in case it hasn't been sorted already
df = df.sort_values(['permno','caldt'])

# Later I will use month date to bin since it's possible (but highly unlikely) that different stocks have different month ending dates
df['mdt'] = df['caldt'].dt.strftime("%Y-%m")

# It's important to reset the index after cleaning
df = df.reset_index(drop=True)

# You can use this line here to play with how far back the data goes
# df = df[df['caldt'] >= '2000-01-01']

df.tail()

,caldt,permno,beme,beme_adj,prc,ret,mdt
3534200,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08
3534201,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09
3534202,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10
3534203,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11
3534204,2023-12-31,93436.0,0.114912,-0.731231,248.479996,0.034988,2023-12


In [102]:
# We need to lag any variables that we use to form our portfolios
df['beme_lag'] = df.groupby('permno')['beme'].shift(1)

# It's common practice to filter out low priced stocks (less than $5 per share)
df['prc_lag'] = df.groupby('permno')['prc'].shift(1)

# We also filter out any rows that don't have our binning variable
df = df.query("beme_lag == beme_lag and prc_lag >= 5").reset_index(drop=True)

df.tail()

,caldt,permno,beme,beme_adj,prc,ret,mdt,beme_lag,prc_lag
2071643,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08,0.114912,267.429993
2071644,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09,0.114912,258.079987
2071645,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10,0.114912,250.220001
2071646,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11,0.114912,200.839996
2071647,2023-12-31,93436.0,0.114912,-0.731231,248.479996,0.034988,2023-12,0.114912,240.080002


In [103]:
# This is where the magic happens
# We group the data on the month level and then sort the rows into 10 equally large bins based on book to market value
# 0 is low beme while 9 is high bme
df['beme_bins'] = df.groupby("mdt")['beme_lag'].transform(lambda x: pd.qcut(x, 10, labels=False))

df.tail()

,caldt,permno,beme,beme_adj,prc,ret,mdt,beme_lag,prc_lag,beme_bins
2071643,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08,0.114912,267.429993,0
2071644,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09,0.114912,258.079987,0
2071645,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10,0.114912,250.220001,0
2071646,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11,0.114912,200.839996,0
2071647,2023-12-31,93436.0,0.114912,-0.731231,248.479996,0.034988,2023-12,0.114912,240.080002,0


In [104]:
# We then create a portfolio dataframe that consists of 10 equally weighted beme portfolios
# Again, portfolio 0 is low beme while portfolio 9 is high beme
port = df.groupby(['caldt', 'beme_bins'])['ret'].mean().unstack(level=['beme_bins'])*100

# Drop rows from port where we don't know the holding return (in this case the most recent month)
port = port.dropna()

# We create a long short portfolio where we long the high beme portfolio and short the low beme portfolio
# This portfolio is called zero-cost since it has not initial outlay
# It will also be closer to market neutral (beta of 0) and have higher returns than the other portfolios on average
port['spread'] = port[9] - port[0]

port.tail()

beme_bins,0,1,2,3,4,5,6,7,8,9,spread
caldt,,,,,,,,,,,
2023-08-31,-4.849009,-4.574571,-4.926846,-4.349804,-3.902073,-5.526461,-4.530687,-5.589370,-3.378024,-4.060495,0.788514
2023-09-30,-6.653298,-6.050211,-6.849428,-5.238536,-5.863064,-5.509421,-5.791091,-5.267759,-4.304507,-5.641488,1.011809
2023-10-31,-6.801395,-7.252412,-7.490120,-7.098177,-6.117007,-5.061635,-4.371535,-4.607268,-6.686144,-6.658403,0.142992
2023-11-30,12.086922,10.718124,9.153909,8.771951,8.918538,9.885827,10.012618,8.950096,8.381541,9.365126,-2.721796
2023-12-31,9.944934,9.574295,12.236820,10.447453,10.038035,12.604399,12.546713,12.888091,14.410244,12.093928,2.148994


In [105]:
# This is a nice function that some BYU quant professors made to look at the performance of portfolios
# The mean value here would be the average 1 month return for each portfolio over the history of our dataset
# The other important value is the tstat. Generally a tstat of 2 or higher (equivalent to p-value < .05) is statistically significant.
# We see that all of the portfolios are statistically significant and that our spread portfolio has a high return (.8% per month ~ 9.6% annually)
# and is also statistically significant! Note that if our portfolio has a beta of 0 then this would be 9.6% alpha which is super high (so something might be fishy)
summary(port).loc[['count','mean','std','tstat']].round(3)

beme_bins,0,1,2,3,4,5,6,7,8,9,spread
count,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000
mean,0.645,0.819,0.966,1.032,1.091,1.155,1.190,1.202,1.341,1.448,0.803
std,6.718,6.068,5.606,5.414,5.239,5.059,4.903,4.960,4.877,5.380,4.224
tstat,2.731,3.838,4.900,5.421,5.923,6.492,6.901,6.895,7.820,7.657,5.409
